This notebooks show the implememtation of our DCESumm approach that has been proposed in our paper. For the illustration purpose, we consider 50 random US Test document-summary pairs for the experimental purpose.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.9 MB/s eta 0:00:00


In [ ]:
from time import time
import numpy as np
import tensorflow as tf
import copy
import statistics
import re
import random
from nltk import sent_tokenize
import nltk
nltk.download('punkt')
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer, InputSpec, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import callbacks
from tensorflow.keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
import pandas as pd
from numpy import float32
from keras import initializers
np.random.seed(10)
tf.random.set_seed(10)
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, AutoModel

import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not, use cpu
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
US=[]
f= open("/content/drive/My Drive/Colab Notebooks/us_bill_test/cleantext.txt","r")
f1= f.readlines()
for i in range(len(f1)):
  f1[i]= f1[i].replace('\n','')
  US.append(f1[i])
    
newUS=[]
for i in range(len(US)):
  newUS.append(US[i].replace("<SECTION-HEADER>",""))
    
for i in range(len(newUS)):
  newUS[i]=newUS[i].lower()
    
    
US_Summary=[]
f= open("/content/drive/My Drive/Colab Notebooks/us_bill_test/cleansummary.txt","r")
f1= f.readlines()
for i in range(len(f1)):
    f1[i]= f1[i].replace('\n','')
    US_Summary.append(f1[i])
    
for i in range(len(US_Summary)):
  US_Summary[i]= US_Summary[i].lower()

In [ ]:
USTestList=[]
USActualSum=[]
for i in range(len(US_Summary)):
  if (len(sent_tokenize(newUS[i])))>10:
    USTestList.append(newUS[i])
    USActualSum.append(US_Summary[i])

In [ ]:
newUS= copy.deepcopy(USTestList)
US_Summary=copy.deepcopy(USActualSum)

In [ ]:
randInd=random.sample(range(len(newUS)),50)

In [ ]:
smallnewUS=[]
smallUSSummary=[]
for index in randInd:
  smallnewUS.append(newUS[index])
  smallUSSummary.append(US_Summary[index])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
bert_model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased",output_hidden_states=True)
bert_model = bert_model.to(device)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
keras_model= load_model('/content/drive/My Drive/Colab Notebooks/model_1278177.h5') #loading the saved MLP model from the drive

In [ ]:
def bert_embeds(doc):
    test_sent_list= sent_tokenize(doc)
    np_tensor=[]
    for i in range(len(test_sent_list)):
        granola_ids = tokenizer.encode(test_sent_list[i], max_length=100)
        granola_ids = torch.LongTensor(granola_ids)
        granola_ids = granola_ids.to(device)
        bert_model.eval()
        granola_ids = granola_ids.unsqueeze(0)
        with torch.no_grad():
            out = bert_model(input_ids=granola_ids)
        hidden_states = out[2]
        sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
        np_tensor.append(sentence_embedding.cpu().numpy())
        
    return np_tensor

In [ ]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img= Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

In [ ]:
dims = [768, 64, 128, 256, 5]
init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
autoencoder, encoder = autoencoder(dims, init=init)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


## Build clustering model

In [ ]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(name='clusters',shape=(self.n_clusters, input_dim), initializer='glorot_uniform')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [ ]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [ ]:
def predictions(Doc,item, n_clusters):
  sentList=sent_tokenize(Doc)
  DocEmbeds= bert_embeds(Doc)
  DocEmbeds= np.array(DocEmbeds)
  MLPScores= keras_model.predict(DocEmbeds)
  MLPScores= list(MLPScores.ravel())
  #defining hyperparameters
  pretrain_optimizer = SGD(lr=0.01, momentum=0.9)
  pretrain_epochs = 100
  batch_size = 5
  #pretrain autoencoder
  autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
  autoencoder.fit(DocEmbeds, DocEmbeds, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)
  kmeans = KMeans(n_clusters=n_clusters, n_init=20) #initialize the clusteting
  y_pred = kmeans.fit_predict(encoder.predict(DocEmbeds))
  y_pred_last= np.copy(y_pred)
  clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
  cluster_model = Model(inputs=encoder.input, outputs=clustering_layer)
  cluster_model.compile(optimizer=SGD(0.01, 0.9), loss='kld')
  # print(cluster_model.summary())

  loss = 0
  index = 0
  maxiter = 8000
  update_interval = 140
  index_array = np.arange(DocEmbeds.shape[0])
  tol = 0.001 # tolerance threshold to stop training
  for ite in range(int(maxiter)):
      if ite % update_interval == 0:
          q = cluster_model.predict(DocEmbeds, verbose=0)
          p = target_distribution(q)  # update the auxiliary target distribution p

          # evaluate the clustering performance
          y_pred = q.argmax(1)

          # check stop criterion - model convergence
          delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
          y_pred_last = np.copy(y_pred)
          if ite > 0 and delta_label < tol:
              print('delta_label ', delta_label, '< tol ', tol)                                                     
              print('Reached tolerance threshold. Stopping training.')
              break
      idx = index_array[index * batch_size: min((index+1) * batch_size, DocEmbeds.shape[0])]
      loss = cluster_model.train_on_batch(x=DocEmbeds[idx], y=p[idx])
      index = index + 1 if (index + 1) * batch_size <= DocEmbeds.shape[0] else 0

  y_pred_last = np.copy(y_pred)
  cluster_model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

  q = cluster_model.predict(DocEmbeds, verbose=0)
  p = target_distribution(q)  # update the auxiliary target distribution p
  Clusters= q.argmax(1)
  Clusters= y_pred.tolist() # formation of clusters
  
  clusterDict = {} 
  for i in range(len(y_pred)):
    if y_pred[i] not in clusterDict:
      tempList = []
      tempList.append((i, sentList[i]))
      clusterDict[y_pred[i]] = copy.deepcopy(tempList)
    else:
      clusterDict[y_pred[i]].append((i,sentList[i]))
  # Get individual scores
  indScores = []
  for key in clusterDict:
    tempScores = []
    for i in range(len(clusterDict[key])):
      tempScores.append((clusterDict[key][i][0], MLPScores[clusterDict[key][i][0]]))
    indScores.append(tempScores)

  # Get cluster wise quality scores
  clsQuality = []
  for i in range(len(indScores)):
    clusterKScores = []
    for j in range(len(indScores[i])):
      clusterKScores.append(indScores[i][j][1])
    clsQuality.append(statistics.median(clusterKScores))

  # Update individual scores with cluster quality scores
  for i in range(len(indScores)):
    for j in range(len(indScores[i])):
      indScores[i][j] = (indScores[i][j][0], ((clsQuality[i] * indScores[i][j][1]) + indScores[i][j][1])/2)
  allSentScores = []
  for i in range(len(indScores)):
    for j in range(len(indScores[i])):
      allSentScores.append((indScores[i][j][0], indScores[i][j][1]))

  allSentScores.sort(key = lambda x: x[1], reverse=True) #sorting the scores in increasing order
  sorted_index=[] 
  for i in range(len(allSentScores)):
    sorted_index.append(allSentScores[i][0])
  top_n= int(len(sorted_index)*0.15) #pick 15% of the top scoring sentences for the summary formation
  selected_index= sorted_index[:top_n]

  PredSum=[]
  for i in range(len(selected_index)):
    PredSum.append(sentList[selected_index[i]])
  final_summary=''.join(PredSum)
  final_summary=re.sub(r"\.(?=\S)", ". ", final_summary)
  end_time4=time()
  return final_summary

In [ ]:
PredSumm=[]
for i in range(len(smallnewUS)):
  n_clusters=int(len(sent_tokenize(smallnewUS[i]))*0.35)
  if len(sent_tokenize(smallnewUS[i])) > n_clusters:
    PredSumm.append(predictions(smallnewUS[i],i,n_clusters))
    print('Document traversing: ', str(i))

3/3 [==============================] - 0s 4ms/step


Epoch 1/100
16/16 [==============================] - 1s 4ms/step - loss: 0.0230
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 0.0202
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0195
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0191
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0188
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0185
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0183
Epoch 8/100
16/16 [==============================] - 0s 4ms/step - loss: 0.0181
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0179
Epoch 10/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0177
Epoch 11/100
16/16 [==============================] - 0s 4ms/step - loss: 0.0176
Epoch 12/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0174
Epoch 13/100
16/16 [=================

Epoch 1/100
4/4 [==============================] - 1s 10ms/step - loss: 0.0252
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0246
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0235
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0223
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0214
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0207
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0196
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0187
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0180
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0174
Epoch 11/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0170
Epoch 12/100
4/4 [==============================] - 0s 4ms/step - loss: 0.0167
Epoch 13/100
4/4 [==============================] - 0s 4ms/s

Epoch 1/100
26/26 [==============================] - 2s 7ms/step - loss: 0.0255
Epoch 2/100
26/26 [==============================] - 0s 7ms/step - loss: 0.0206
Epoch 3/100
26/26 [==============================] - 0s 6ms/step - loss: 0.0197
Epoch 4/100
26/26 [==============================] - 0s 6ms/step - loss: 0.0194
Epoch 5/100
26/26 [==============================] - 0s 7ms/step - loss: 0.0191
Epoch 6/100
26/26 [==============================] - 0s 6ms/step - loss: 0.0189
Epoch 7/100
26/26 [==============================] - 0s 6ms/step - loss: 0.0187
Epoch 8/100
26/26 [==============================] - 0s 5ms/step - loss: 0.0186
Epoch 9/100
26/26 [==============================] - 0s 4ms/step - loss: 0.0185
Epoch 10/100
26/26 [==============================] - 0s 4ms/step - loss: 0.0183
Epoch 11/100
26/26 [==============================] - 0s 4ms/step - loss: 0.0183
Epoch 12/100
26/26 [==============================] - 0s 4ms/step - loss: 0.0181
Epoch 13/100
26/26 [=================

Epoch 1/100
11/11 [==============================] - 2s 7ms/step - loss: 0.0599
Epoch 2/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0387
Epoch 3/100
11/11 [==============================] - 0s 5ms/step - loss: 0.0307
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0283
Epoch 5/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0259
Epoch 6/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0245
Epoch 7/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0233
Epoch 8/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0226
Epoch 9/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0220
Epoch 10/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0217
Epoch 11/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0214
Epoch 12/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0212
Epoch 13/100
11/11 [=================

...

Epoch 1/100
8/8 [==============================] - 2s 6ms/step - loss: 0.0311
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0302
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0289
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0279
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0268
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0261
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0253
Epoch 8/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0248
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0243
Epoch 10/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0240
Epoch 11/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0236
Epoch 12/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0232
Epoch 13/100
8/8 [==============================] - 0s 5ms/st

Epoch 1/100
21/21 [==============================] - 1s 5ms/step - loss: 0.0230
Epoch 2/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0214
Epoch 3/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0207
Epoch 4/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0202
Epoch 5/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0197
Epoch 6/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0194
Epoch 7/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0191
Epoch 8/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0188
Epoch 9/100
21/21 [==============================] - 0s 5ms/step - loss: 0.0186
Epoch 10/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0184
Epoch 11/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0181
Epoch 12/100
21/21 [==============================] - 0s 4ms/step - loss: 0.0180
Epoch 13/100
21/21 [=================

Epoch 1/100
8/8 [==============================] - 1s 5ms/step - loss: 0.0204
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0201
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0199
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0197
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0194
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0192
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0190
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0189
Epoch 9/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0187
Epoch 10/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0186
Epoch 11/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0185
Epoch 12/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0183
Epoch 13/100
8/8 [==============================] - 0s 6ms/st

Epoch 1/100
19/19 [==============================] - 1s 5ms/step - loss: 0.0324
Epoch 2/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0241
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0218
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0212
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0208
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0206
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0203
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0201
Epoch 9/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0199
Epoch 10/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0197
Epoch 11/100
19/19 [==============================] - 0s 6ms/step - loss: 0.0195
Epoch 12/100
19/19 [==============================] - 0s 6ms/step - loss: 0.0193
Epoch 13/100
19/19 [=================

Epoch 1/100
20/20 [==============================] - 2s 7ms/step - loss: 0.0244
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 0.0215
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.0209
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.0205
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.0202
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0199
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0197
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0195
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0193
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0191
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0190
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0188
Epoch 13/100
20/20 [=================

In [ ]:
!pip install rouge
from rouge import Rouge
rouge=Rouge()
print(rouge.get_scores(PredSumm, smallUSSummary, avg=True))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
{'rouge-1': {'r': 0.6011233466344388, 'p': 0.3661056087527077, 'f': 0.4322654947081938}, 'rouge-2': {'r': 0.38834766507668983, 'p': 0.20543011820158694, 'f': 0.24884139198909128}, 'rouge-l': {'r': 0.5517225032836396, 'p': 0.33409856273206545, 'f': 0.3952631451909059}}


In [ ]:
{'rouge-1': {'r': 0.6011233466344388, 'p': 0.3661056087527077, 'f': 0.4322654947081938}, 
 'rouge-2': {'r': 0.38834766507668983, 'p': 0.20543011820158694, 'f': 0.24884139198909128}, 
 'rouge-l': {'r': 0.5517225032836396, 'p': 0.33409856273206545, 'f': 0.3952631451909059}}
